## CODE REVIEW 
#### Milling Tool Wear Maintenance Policy using the REINFORCE algorithm

In [19]:
import numpy as np
import pandas as pd
import stable_baselines3 as sb3
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C, PPO, DQN

from milling_tool_environment import MillingTool_SS_NT
from utilities import compute_metrics, compute_metrics_simple, test_script, write_test_results, downsample
from reinforce_classes import PolicyNetwork, Agent

In [5]:
df_expts = pd.read_csv('Experiments.csv')
n_expts = len(df_expts.index)
n_expt = 0
experiment_summary = []
print(df_expts.iloc[0])

expt_n                                                                    0
environment                                                              SS
environment_info                                       Simulated Dasic 2006
data_file                 data\Simulated_Dasic_2006_Tool_Wear_Model_Trai...
Model_name                                         RF_Model_Dasic_NoNBD.mdl
environment.1                                                            SS
R1                                                                        1
R2                                                                       -1
R3                                                                     -100
wear_threshold                                                          3.0
threshold_factor                                                        1.0
add_noise                                                                 0
breakdown_chance                                                        0.0
episodes    

In [6]:
# Load experiment parameters
ENVIRONMENT_CLASS = df_expts['environment'][n_expt]
DATA_FILE = df_expts['data_file'][n_expt]
R1 = df_expts['R1'][n_expt]
R2 = df_expts['R2'][n_expt]
R3 = df_expts['R3'][n_expt]
WEAR_THRESHOLD = df_expts['wear_threshold'][n_expt]
THRESHOLD_FACTOR = df_expts['threshold_factor'][n_expt]
ADD_NOISE = df_expts['add_noise'][n_expt]
BREAKDOWN_CHANCE = df_expts['breakdown_chance'][n_expt]
EPISODES = df_expts['episodes'][n_expt]
MILLING_OPERATIONS_MAX = df_expts['milling_operations_max'][n_expt]
TEST_CASES = df_expts['test_cases'][n_expt]
TEST_ROUNDS = df_expts['test_rounds'][n_expt]

RESULTS_FOLDER = df_expts['results_folder'][n_expt]
TEST_FILE = df_expts['test_file'][n_expt]
TRAIN_SR = df_expts['train_sample_rate'][n_expt]
TEST_SR = df_expts['test_sample_rate'][n_expt]

METRICS_METHOD = 'binary'
WEAR_THRESHOLD_NORMALIZED = 0.0

# EPISODES=20
# MILLING_OPERATIONS_MAX=10

print(f'Episodes: {EPISODES} x {MILLING_OPERATIONS_MAX}')

Episodes: 800 x 121


### Read tool wear data

In [7]:
df = pd.read_csv(DATA_FILE)
n_records = len(df.index)

df['ACTION_CODE'] = np.where(df['tool_wear'] < WEAR_THRESHOLD, 0.0, 1.0)

# 3. Normalize
WEAR_MIN = df['tool_wear'].min()
WEAR_MAX = df['tool_wear'].max()
WEAR_THRESHOLD_ORG_NORMALIZED = (WEAR_THRESHOLD-WEAR_MIN)/(WEAR_MAX-WEAR_MIN)
WEAR_THRESHOLD_NORMALIZED = THRESHOLD_FACTOR*(WEAR_THRESHOLD-WEAR_MIN)/(WEAR_MAX-WEAR_MIN)
df_normalized = (df-df.min())/(df.max()-df.min())
df_normalized['ACTION_CODE'] = df['ACTION_CODE']
print(f'- Tool wear data imported ({len(df.index)} records).')

# 4. Split into train and test
df_train = downsample(df_normalized, TRAIN_SR)
df_train.to_csv('TempTrain.csv')
df_train = pd.read_csv('TempTrain.csv')

df_test = downsample(df_normalized, TEST_SR)
df_test.to_csv('TempTest.csv')
df_test = pd.read_csv('TempTest.csv')
print(f'- Tool wear data split into train ({len(df_train.index)} records) and test ({len(df_test.index)} records).')

- Tool wear data imported (121 records).
- Down-sampling. Input data records: 121. Sampling rate: 1. Expected rows 121. Down-sampled to 121 rows.
- Down-sampling. Input data records: 121. Sampling rate: 2. Expected rows 60. Down-sampled to 61 rows.
- Tool wear data split into train (121 records) and test (61 records).


In [8]:
env = MillingTool_SS_NT(df_train, WEAR_THRESHOLD_NORMALIZED, MILLING_OPERATIONS_MAX, ADD_NOISE, BREAKDOWN_CHANCE, R1, R2, R3)
env_test = MillingTool_SS_NT(df_test, WEAR_THRESHOLD_NORMALIZED, MILLING_OPERATIONS_MAX, ADD_NOISE, BREAKDOWN_CHANCE, R1, R2, R3)

** -- Single-variate env. Terminate on (1) tool breakdown (2) data-end (3) milling operations end.  R1: 1, R2: -1, R3: -100. Noise: 0. Break-down chance: 0.0 -- **
** -- Single-variate env. Terminate on (1) tool breakdown (2) data-end (3) milling operations end.  R1: 1, R2: -1, R3: -100. Noise: 0. Break-down chance: 0.0 -- **


### Train REINFORCE

In [11]:
# Policy network learning parameters
gamma = 0.99
alpha = 0.01

rewards_history = []
loss_history = []

input_dim = env.observation_space.shape[0]
output_dim = env.action_space.n

agent_RF = Agent(input_dim, output_dim, alpha, gamma)

for episode in range(EPISODES):
    state = env.reset()

    # Sample a trajectory
    for t in range(MILLING_OPERATIONS_MAX): # Max. milling operations desired
        action = agent_RF.act(state)
        state, reward, done, info = env.step(action)
        agent_RF.rewards.append(reward)
        if done:
            break

    # Learn during this episode
    loss = agent_RF.learn() # train per episode
    total_reward = sum(agent_RF.rewards)

    # Record statistics for this episode
    rewards_history.append(total_reward)
    loss_history.append(loss.item()) # Extract values from list of torch items for plotting

    # On-policy - so discard all data
    agent_RF.onpolicy_reset()

    if (episode%100 == 0):
        # print(f'[{episode:04d}] Loss: {loss:>10.2f} | Reward: {total_reward:>10.2f} | Ep.length: {env.ep_length:04d}')
        print(f'[{episode:04d}] Loss: {loss:>10.2e} | Reward: {total_reward:>10.2e} | Ep.length: {env.ep_length:04d}')

[0000] Loss:  -6.60e+01 | Reward:  -1.10e+00 | Ep.length: 0348
[0100] Loss:   3.96e+00 | Reward:   2.30e+00 | Ep.length: 0348
[0200] Loss:   4.82e+00 | Reward:   3.27e+00 | Ep.length: 0348
[0300] Loss:   8.80e+00 | Reward:   4.70e+00 | Ep.length: 0348
[0400] Loss:   7.38e+00 | Reward:   4.16e+00 | Ep.length: 0348
[0500] Loss:   9.55e+00 | Reward:   3.92e+00 | Ep.length: 0348
[0600] Loss:   5.79e+00 | Reward:   4.54e+00 | Ep.length: 0348
[0700] Loss:   3.66e+00 | Reward:   3.46e+00 | Ep.length: 0348


### Test

In [9]:
idx_replace_cases = df_test.index[df_test['ACTION_CODE'] >= 1.0]
idx_normal_cases = df_test.index[df_test['ACTION_CODE'] < 1.0]

avg_Pr = avg_Rc = avg_F1 = 0.0

# Create test cases
idx_replace_cases = np.random.choice(idx_replace_cases, int(TEST_CASES/2), replace=False)
idx_normal_cases = np.random.choice(idx_normal_cases, int(TEST_CASES/2), replace=False)
test_cases = [*idx_normal_cases, *idx_replace_cases]

results = test_script(METRICS_METHOD, 1, df_test, 'REINFORCE', EPISODES, env_test, 'ENVIRONMENT_INFO', agent_RF,
                      test_cases, 'TEST_INFO', DATA_FILE, WEAR_THRESHOLD, 'RESULTS_FILE.csv')
avg_Pr, avg_Rc, avg_F1 = results[14:17]
print(f'- RF:  Pr: {avg_Pr:0.3f} \t Rc: {avg_Rc:0.3f} \t F1:{avg_F1:0.3f}')

NameError: name 'agent_RF' is not defined

### Stable-Baselines Algorithms

## Network architectures for Stable-Baselines Algorithms

In [ ]:
env = MillingTool_SS_NT(df_train, WEAR_THRESHOLD_NORMALIZED, MILLING_OPERATIONS_MAX, ADD_NOISE, BREAKDOWN_CHANCE, R1, R2, R3)

In [24]:
agent_SB = A2C('MlpPolicy', env)
print(120*'-')
print(agent_SB.policy)

print(120*'-')
agent_SB = DQN('MlpPolicy', env)
print(agent_SB.policy)

print(120*'-')
agent_SB = PPO('MlpPolicy', env)
print(agent_SB.policy)

------------------------------------------------------------------------------------------------------------------------
ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (shared_net): Sequential()
    (policy_net): Sequential(
      (0): Linear(in_features=2, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=2, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=2, bias=True)
  (value_net): Linear(in_features=64, out_feature

In [20]:
sb3.common.policies.ActorCriticPolicy.net_arch

AttributeError: type object 'ActorCriticPolicy' has no attribute 'net_arch'

In [13]:
env = MillingTool_SS_NT(df_train, WEAR_THRESHOLD_NORMALIZED, MILLING_OPERATIONS_MAX, ADD_NOISE, BREAKDOWN_CHANCE, R1, R2, R3)
agent_SB = A2C('MlpPolicy', env, tensorboard_log="./tensorboard/")

agent_SB.learn(total_timesteps=10_000)

# METRICS_METHOD None, 'micro', 'macro', 'weighted', 'samples')
results = test_script(METRICS_METHOD, 1, df_test, 'A2C', EPISODES, env_test, 'ENVIRONMENT_INFO', agent_SB, test_cases, 'TEST_INFO', DATA_FILE, WEAR_THRESHOLD, 'RESULTS_FILE.csv')
avg_Pr, avg_Rc, avg_F1 = results[14:17]
print(f'- A2C: Pr: {avg_Pr:0.3f} \t Rc: {avg_Rc:0.3f} \t F1:{avg_F1:0.3f}')

** -- Single-variate env. Terminate on (1) tool breakdown (2) data-end (3) milling operations end.  R1: 1, R2: -2, R3: -40. Noise: 0. Break-down chance: 0.0 -- **
- A2C: Pr: 0.583 	 Rc: 0.700 	 F1:0.603


In [14]:
env = MillingTool_SS_NT(df_train, WEAR_THRESHOLD_NORMALIZED, MILLING_OPERATIONS_MAX, ADD_NOISE, BREAKDOWN_CHANCE, R1, R2, R3)
agent_SB = DQN('MlpPolicy', env, tensorboard_log="./tensorboard/")
agent_SB.learn(total_timesteps=10_000)

results = test_script(METRICS_METHOD, 1, df_test, 'DQN', EPISODES, env_test, 'ENVIRONMENT_INFO', agent_SB, test_cases, 'TEST_INFO', DATA_FILE, WEAR_THRESHOLD, 'RESULTS_FILE.csv')
avg_Pr, avg_Rc, avg_F1 = results[14:17]
print(f'- DQN: Pr: {avg_Pr:0.3f} \t Rc: {avg_Rc:0.3f} \t F1:{avg_F1:0.3f}')

** -- Single-variate env. Terminate on (1) tool breakdown (2) data-end (3) milling operations end.  R1: 1, R2: -2, R3: -40. Noise: 0. Break-down chance: 0.0 -- **
- DQN: Pr: 0.000 	 Rc: 0.000 	 F1:0.000


In [15]:
env = MillingTool_SS_NT(df_train, WEAR_THRESHOLD_NORMALIZED, MILLING_OPERATIONS_MAX, ADD_NOISE, BREAKDOWN_CHANCE, R1, R2, R3)
agent_SB = PPO('MlpPolicy', env, tensorboard_log="./tensorboard/")
agent_SB.learn(total_timesteps=10_000)

results = test_script(METRICS_METHOD, 1, df_test, 'PPO', EPISODES, env_test, 'ENVIRONMENT_INFO', agent_SB, test_cases, 'TEST_INFO', DATA_FILE, WEAR_THRESHOLD, 'RESULTS_FILE.csv')
avg_Pr, avg_Rc, avg_F1 = results[14:17]
print(f'- PPO: Pr: {avg_Pr:0.3f} \t Rc: {avg_Rc:0.3f} \t F1:{avg_F1:0.3f}')

** -- Single-variate env. Terminate on (1) tool breakdown (2) data-end (3) milling operations end.  R1: 1, R2: -2, R3: -40. Noise: 0. Break-down chance: 0.0 -- **
- PPO: Pr: 0.565 	 Rc: 0.650 	 F1:0.580


### Records

```
Run-1: 800 eps
- A2C: Pr: 0.455 	 Rc: 0.250 	 F1:0.391
- DQN: Pr: 0.000 	 Rc: 0.000 	 F1:0.000
- PPO: Pr: 0.333 	 Rc: 0.150 	 F1:0.268
----------------------------------------------------
- RF:  Pr: 0.800 	 Rc: 1.000 	 F1:0.833
```

# Consolidate multiple (3) test runs data and find averaged metrics 

- Metrics file RF_performance_summary is created using ```compute_metrics``` function
- This fn uses ```df.groupby(['Algorithm']).agg({'Precision': ['mean','std']```

In [9]:
import pandas as pd
import glob
# from utilities import compute_metrics

In [10]:
# PATH = './results/MultiRound' 
# REPORTS = f'{PATH}/R?_RF_performance_summary.csv'
# METRICS = f'{PATH}/R?_TEST_CONSOLIDATED_METRICS.csv'

# CONCAT_PERFORMANCE_REPORT = f'{PATH}/Concat_RF_performance_summary_21-Jun-2023.csv'
# CONCAT_METRICS = f'{PATH}/Concat_Metrics_21-Jun-2023.csv'
# CONSOLIDATED_PERFORMANCE_REPORT = f'{PATH}/Consolidated_RF_performance_summary_21-Jun-2023.csv'

# report_files = glob.glob(REPORTS) 
# print(report_files)

# df_report = pd.concat((pd.read_csv(f, header = 0) for f in report_files))
# df_report.to_csv(CONCAT_PERFORMANCE_REPORT)

# df_m = pd.concat((pd.read_csv(f, header = 0) for f in metrics_files))
# df_m.to_csv(CONCAT_METRICS)

In [11]:
# For multi-training round - stability test experiments 
PATH = './results/MultiRound' 
EXPT_REPORTS = f'{PATH}/Experiment_Results_*.csv'
CONCAT_REPORT = f'{PATH}/Concat_Experiments.csv'
CONSOLIDATED_PERFORMANCE_REPORT = f'{PATH}/Consolidated_Experiments_Summary.csv'

report_files = glob.glob(EXPT_REPORTS) 
print(report_files)

df_report = pd.concat((pd.read_csv(f, header = 0) for f in report_files))
df_report.to_csv(CONCAT_REPORT)

['./results/MultiRound\\Experiment_Results_0_14 - Copy.csv', './results/MultiRound\\Experiment_Results_0_14.csv']


In [12]:
df_report.columns

Index(['Unnamed: 0', 'expt_n', 'environment', 'environment_info', 'data_file',
       'Model_name', 'environment.1', 'R1', 'R2', 'R3', 'wear_threshold',
       'threshold_factor', 'add_noise', 'breakdown_chance', 'episodes',
       'milling_operations_max', 'version_prefix', 'test_info', 'test_cases',
       'test_rounds', 'results_folder', 'test_file', 'train_sample_rate',
       'test_sample_rate', 'RF_Pr', 'RF_Rc', 'RF_F1', 'RF_F05', 'RF_Pr_sd',
       'RF_Rc_sd', 'RF_F1_sd', 'RF_F05_sd', 'A2C_Pr', 'A2C_Rc', 'A2C_F1',
       'A2C_F05', 'A2C_Pr_sd', 'A2C_Rc_sd', 'A2C_F1_sd', 'A2C_F05_sd',
       'DQN_Pr', 'DQN_Rc', 'DQN_F1', 'DQN_F05', 'DQN_Pr_sd', 'DQN_Rc_sd',
       'DQN_F1_sd', 'DQN_F05_sd', 'PPO_Pr', 'PPO_Rc', 'PPO_F1', 'PPO_F05',
       'PPO_Pr_sd', 'PPO_Rc_sd', 'PPO_F1_sd', 'PPO_F05_sd', 'model_file',
       'RF_time', 'A2C_time', 'DQN_time', 'PPO_time'],
      dtype='object')

In [13]:
def compute_aggregated_metrics(df):
    metrics = df.groupby(['environment_info']).agg(
        {'RF_Pr': ['mean'], 'RF_Rc': ['mean'],'RF_F1': ['mean'], 'RF_F05': ['mean'],
         'A2C_Pr': ['mean'], 'A2C_Rc': ['mean'],'A2C_F1': ['mean'], 'A2C_F05': ['mean'],
         'DQN_Pr': ['mean'], 'DQN_Rc': ['mean'],'DQN_F1': ['mean'], 'DQN_F05': ['mean'],
         'PPO_Pr': ['mean'], 'PPO_Rc': ['mean'],'PPO_F1': ['mean'], 'PPO_F05': ['mean'],
         
         'RF_Pr_sd': ['mean'], 'RF_Rc_sd': ['mean'],'RF_F1_sd': ['mean'], 'RF_F05_sd': ['mean'],
         'A2C_Pr_sd': ['mean'], 'A2C_Rc_sd': ['mean'],'A2C_F1_sd': ['mean'], 'A2C_F05_sd': ['mean'],
         'DQN_Pr_sd': ['mean'], 'DQN_Rc_sd': ['mean'],'DQN_F1_sd': ['mean'], 'DQN_F05_sd': ['mean'],
         'PPO_Pr_sd': ['mean'], 'PPO_Rc_sd': ['mean'],'PPO_F1_sd': ['mean'], 'PPO_F05_sd': ['mean']
        
        })
    return(metrics)

In [14]:
algo_metrics = compute_aggregated_metrics(df_report)
algo_metrics.to_csv(CONSOLIDATED_PERFORMANCE_REPORT)

In [15]:
algo_metrics

,RF_Pr,RF_Rc,RF_F1,RF_F05,A2C_Pr,A2C_Rc,A2C_F1,A2C_F05,DQN_Pr,DQN_Rc,...,A2C_F1_sd,A2C_F05_sd,DQN_Pr_sd,DQN_Rc_sd,DQN_F1_sd,DQN_F05_sd,PPO_Pr_sd,PPO_Rc_sd,PPO_F1_sd,PPO_F05_sd
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,...,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
environment_info,,,,,,,,,,,,,,,,,,,,,
PHM C01 MS NBD,0.000000,0.000000,0.000000,0.000000,0.489548,0.495000,0.490043,0.489209,0.400000,0.040000,...,0.127893,0.122916,0.438854,0.045947,0.080383,0.147723,0.128513,0.091439,0.112413,0.126855
PHM C01 SS HBD,0.939417,0.675000,0.782742,0.868716,0.506589,0.795000,0.617987,0.545842,0.266667,0.025000,...,0.062370,0.053340,0.438854,0.042492,0.076038,0.145879,0.149762,0.073786,0.096348,0.120821
PHM C01 SS LBD,0.909307,0.675000,0.772607,0.848399,0.000000,0.000000,0.000000,0.000000,0.769343,0.980000,...,0.000000,0.000000,0.028035,0.025820,0.022296,0.025550,0.139466,0.106066,0.117633,0.128711
PHM C01 SS NBD,0.992857,0.570000,0.720351,0.860303,0.546543,0.430000,0.475526,0.513566,0.225000,0.030000,...,0.076350,0.080070,0.208055,0.025820,0.045323,0.083817,0.083112,0.082496,0.079150,0.078769
PHM C04 MS NBD,0.777687,0.525000,0.621664,0.704391,0.495882,0.845000,0.624475,0.540334,0.498650,0.975000,...,0.030681,0.025620,0.014239,0.035355,0.020239,0.016136,0.050125,0.103280,0.068401,0.056139
PHM C04 SS HBD,0.732020,0.925000,0.815640,0.763023,0.493257,0.520000,0.504165,0.497099,0.501353,0.975000,...,0.098753,0.087801,0.018237,0.042492,0.023877,0.019941,0.184610,0.069921,0.100034,0.135782
PHM C04 SS LBD,0.618073,0.985000,0.759316,0.667729,0.523310,0.520000,0.517009,0.519557,0.616667,0.055000,...,0.087124,0.077895,0.458460,0.049721,0.087039,0.159829,0.207126,0.031623,0.053580,0.093234
PHM C04 SS NBD,0.500000,1.000000,0.666667,0.555556,0.428557,0.335000,0.372231,0.402715,0.500000,0.985000,...,0.081352,0.074049,0.008547,0.024152,0.011985,0.009536,0.325660,0.074722,0.118463,0.187307


- Compute over all avgs.
- Simulated avgs.
- PHM SS
- PHM MS

In [16]:
# Get environment names
envs = df_report.environment_info.unique()
print(envs)

['Simulated Dasic 2006' 'PHM C01 SS NBD' 'PHM C01 SS LBD' 'PHM C01 SS HBD'
 'PHM C04 SS NBD' 'PHM C04 SS LBD' 'PHM C04 SS HBD' 'PHM C06 SS NBD'
 'PHM C06 SS LBD' 'PHM C06 SS HBD' 'PHM C01 MS NBD' 'PHM C04 MS NBD'
 'PHM C06 MS NBD']


In [2]:
import glob
import pandas as pd

# For multi-training round - stability test experiments 
PATH = './results/MultiRound' 
EXPT_REPORTS = f'{PATH}/Experiment_Results_*.csv'
CONCAT_REPORT = f'{PATH}/Concat_Experiments.csv'
CONSOLIDATED_PERFORMANCE_REPORT = f'{PATH}/Consolidated_Experiments_Summary.csv'

report_files = glob.glob(EXPT_REPORTS) 
print(report_files)

df_report = pd.concat((pd.read_csv(f, header = 0) for f in report_files))
df_report.to_csv(CONCAT_REPORT)

def compute_aggregated_metrics(df):
    metrics = df.groupby(['environment_info']).agg(
        {'RF_Pr': ['mean'], 'RF_Rc': ['mean'],'RF_F1': ['mean'], 'RF_F05': ['mean'],
         'A2C_Pr': ['mean'], 'A2C_Rc': ['mean'],'A2C_F1': ['mean'], 'A2C_F05': ['mean'],
         'DQN_Pr': ['mean'], 'DQN_Rc': ['mean'],'DQN_F1': ['mean'], 'DQN_F05': ['mean'],
         'PPO_Pr': ['mean'], 'PPO_Rc': ['mean'],'PPO_F1': ['mean'], 'PPO_F05': ['mean'],
         
         'RF_Pr_sd': ['mean'], 'RF_Rc_sd': ['mean'],'RF_F1_sd': ['mean'], 'RF_F05_sd': ['mean'],
         'A2C_Pr_sd': ['mean'], 'A2C_Rc_sd': ['mean'],'A2C_F1_sd': ['mean'], 'A2C_F05_sd': ['mean'],
         'DQN_Pr_sd': ['mean'], 'DQN_Rc_sd': ['mean'],'DQN_F1_sd': ['mean'], 'DQN_F05_sd': ['mean'],
         'PPO_Pr_sd': ['mean'], 'PPO_Rc_sd': ['mean'],'PPO_F1_sd': ['mean'], 'PPO_F05_sd': ['mean']
        
        })
    return(metrics)

algo_metrics = compute_aggregated_metrics(df_report)
algo_metrics.to_csv(CONSOLIDATED_PERFORMANCE_REPORT)

envs = df_report.environment_info.unique()

overall = envs[0:]
simulated_envs = envs[0:3]
phm_ss_envs = envs[3:12]
phm_ms_envs = envs[12:]

overall_metrics = algo_metrics.loc[overall]
overall_means = overall_metrics.mean()
overall_means = overall_means.sort_index(ascending=True)
file = f'{PATH}/overall_means.csv'
overall_means.to_csv(file)

simulated_metrics = algo_metrics.loc[simulated_envs]
simulated_means = simulated_metrics.mean()
simulated_means = simulated_means.sort_index(ascending=True)
file = f'{PATH}/simulated_means.csv'
simulated_means.to_csv(file)

phm_ss_metrics = algo_metrics.loc[phm_ss_envs]
phm_ss_means = phm_ss_metrics.mean()
phm_ss_means = phm_ss_means.sort_index(ascending=True)
file = f'{PATH}/phm_ss_means.csv'
phm_ss_means.to_csv(file)

phm_ms_metrics = algo_metrics.loc[phm_ms_envs]
phm_ms_means = phm_ms_metrics.mean()
phm_ms_means = phm_ms_means.sort_index(ascending=True)
file = f'{PATH}/phm_ms_means.csv'
phm_ms_means.to_csv(file)

['./results/MultiRound\\Experiment_Results_0_14.csv', './results/MultiRound\\Experiment_Results_1_14.csv', './results/MultiRound\\Experiment_Results_2_14.csv', './results/MultiRound\\Experiment_Results_3_14.csv', './results/MultiRound\\Experiment_Results_4_14.csv']
